# Laboratório 3.4 - Notebook do aluno

## Visão geral

Este laboratório é uma continuação dos laboratórios guiados do Módulo 3. 

Neste laboratório, você dividirá os dados em três conjuntos de dados separados:

- *Training Set* (Conjunto de treinamento) - Ele será usado para treinar o modelo.
- *Validation Set* (Conjunto de validação) - Será usado durante o treinamento para validar o modelo.
- *Test Set* (Conjunto de testes) - Será retido e usado para produzir métricas depois que o modelo for treinado. Você usará esse conjunto de dados em um próximo laboratório.

Com os dados divididos, você treinará um modelo XGBoost usando o Amazon SageMaker.


## Introdução ao cenário do negócio

Você trabalha para prestadores de serviços de saúde e quer melhorar a detecção de anomalias em pacientes ortopédicos. 

Você tem a incumbência de resolver esse problema usando machine learning (ML). Você tem acesso a um conjunto de dados que contém seis componentes (features) biomecânicos e um alvo (target) de *normal* (normal) ou *abnormal* (anormal). Você pode usar esse conjunto de dados (dataset) para treinar um modelo de ML para prever se um paciente terá uma anomalia.


## Sobre este conjunto de dados (datset)

Este conjunto de dados biomédicos (dataset) foi criado pelo Dr. Henrique da Mota durante um período de residência médica no Group of Applied Research in Orthopaedics (GARO) do Centre Médico-Chirurgical de Réadaptation des Massues em Lyon, na França. Os dados foram organizados em duas tarefas de classificação diferentes, mas relacionadas. 

A primeira tarefa consiste em classificar os pacientes como pertencentes a uma das três categorias a seguir: 

- *Normal* (Normal) (100 pacientes)
- *Disk Hernia* (Hérnia de disco) (60 pacientes)
- *Spondylolisthesis* (Espondilolistese) (150 pacientes)

Para a segunda tarefa, as categorias *Disk Hernia* (Hérnia de disco) e *Spondylolisthesis* (Espondilolistese) foram mescladas em uma única categoria, rotulada como *abnormal* (anormal). Portanto, a segunda tarefa consiste em classificar os pacientes como pertencentes a uma das categorias: *Normal* (Normal) (100 pacientes) ou *Abnormal* (Anormal) (210 pacientes).


## Informações sobre o atributo:

Cada paciente é representado no conjunto de dados por seis atributos biomecânicos derivados da forma e da orientação da pelve e da coluna lombar (nesta ordem): 

- Incidência pélvica
- Inclinação pélvica
- Ângulo da lordose lombar
- Inclinação sacral
- Raio pélvico
- Grau de espondilolistese

A convenção a seguir é usada para os rótulos de classe: 
- DH (hérnia de disco)
- Espondilolistese (SL)
- Normal (NO) 
- Anormal (AB)


Para obter mais informações sobre esse conjunto de dados, consulte a [página da Web Conjunto de dados de coluna vertebral](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).


## Atribuições do conjunto de dados

Esse conjunto de dados foi obtido de:
Dua, D. e Graff, C. (2019). repositório UCI Machine Learning (http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


# Configuração do laboratório
Como essa solução é dividida em vários laboratórios no módulo, você deve executar as seguintes células para poder carregar os dados.

## Importação de dados

Ao executar as células a seguir, os dados serão importados e estarão prontos para uso. 

**Observação:** As células a seguir representam as principais etapas dos laboratórios anteriores.

In [3]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff
import boto3

In [4]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

In [5]:
data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

In [6]:
class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

# Etapa 1: Explorando os dados
Você começará com uma revisão rápida dos dados contidos no conjunto de dados (dataset).

Para aproveitar ao máximo este laboratório, leia atentamente as instruções e o código antes de executar as células. Reserve um tempo para experimentar!

Primeiro, use a função **shape** (para examinar o número de linhas e colunas).

In [7]:
df.shape

(310, 7)

Em seguida, obtenha uma lista das colunas.

In [8]:
df.columns

Index(['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle',
       'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis', 'class'],
      dtype='object')

É possível ver os seis componentes biomecânicos (features) e que a coluna de alvo (target) é chamada *class* (classe).


# Etapa 2: Preparação dos dados

Para este laboratório, você deve dividir os dados em três conjuntos de dados (datasets)

Uma pesquisa na Internet mostrará muitas maneiras diferentes de dividir conjuntos de dados (datasets) Muitos exemplos de código que você pode encontrar dividirão o conjunto de dados (dataset) em *target* (alvo) e *features* (componentes). Em seguida, eles dividirão cada um desses dois conjuntos de dados (datasets) em três subconjuntos (subsets), o que resulta em um total de seis conjuntos de dados (datsets) para rastrear.

## Como mover a posição da coluna alvo (target)

O XGBoost requer que os dados de treinamento estejam em um único arquivo. O arquivo deve ter o valor de destino como a primeira coluna. 

Obtenha a coluna alvo (target) e mova-a para a primeira posição.

In [10]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

Você deve ver que a **class** (classe) agora é a primeira coluna.

In [11]:
df.columns

Index(['degree_spondylolisthesis', 'class', 'pelvic_incidence', 'pelvic_tilt',
       'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius'],
      dtype='object')

## Divisão de dados

Você começará dividindo o conjunto de dados (datasets) em dois. Você usará um conjunto de dados (dataset) para treinamento e dividirá o outro novamente para uso com validação e teste.

Você usará a função *train_test_split* da biblioteca *scikit-learn*, que é uma biblioteca gratuita de machine learning para Python. Ela tem muitos algoritmos e funções úteis, como o que você usará. 

- Para obter mais informações sobre a função, consulte a [documentação Train_test_split] (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). 
 - Para obter mais informações sobre scikit-learn, consulte o [guia scikit-learn] (https://scikit-learn.org/stable/)

Como você não tem uma grande quantidade de dados, certifique-se de que os conjuntos de dados (datasets) que foram divididos contenham uma quantidade representativa de cada classe. Assim, você usará a opção *stratify* (estratificar). Por fim, você usará um número aleatório para que possa repetir as divisões.

In [16]:
from sklearn.model_selection import train_test_split
train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])

Em seguida, divida o conjunto de dados (dataset) *test_and_validate* em duas partes iguais.

In [17]:
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

Examine os três conjuntos de dados.

In [23]:
print(train.shape)
print(test.shape)
print(validate.shape)
print(train)
print(validate)


(248, 7)
(31, 7)
(31, 7)
     degree_spondylolisthesis  class  pelvic_incidence  pelvic_tilt  \
202                101.218783      1         76.314028    41.933683   
178                 52.467552      1         80.654320    26.344379   
68                   1.010041      1         72.076278    18.946176   
118                 16.378086      1         65.536003    24.157487   
182                 54.965789      1         75.437748    31.539454   
..                        ...    ...               ...          ...   
282                  2.737035      0         53.683380    13.447022   
265                  5.360051      0         48.170746     9.594217   
180                 33.607027      1         37.903910     4.479099   
28                   2.652321      1         44.551012    21.931147   
250                 -2.092507      0         36.157830    -0.810514   

     lumbar_lordosis_angle  sacral_slope  pelvic_radius  
202              93.284863     34.380345     132.267286  
178   

Agora, verifique a distribuição das classes.

In [19]:
print(train['class'].value_counts())
print(test['class'].value_counts())
print(validate['class'].value_counts())

class
1    168
0     80
Name: count, dtype: int64
class
1    21
0    10
Name: count, dtype: int64
class
1    21
0    10
Name: count, dtype: int64


## Como fazer upload dos dados para o Amazon S3

O XGboost carregará os dados para treinamento do Amazon Simple Storage Service (Amazon S3). Portanto, você deve gravar os dados em um arquivo CSV (valores separados por vírgulas) e, em seguida, fazer upload do arquivo para o Amazon S3.

Comece configurando algumas variáveis para o bucket do S3 e, em seguida, crie uma função para fazer upload do arquivo CSV para o Amazon S3. Você pode reutilizar essa função.

Primeiro, explore a função.

Observe a seguinte linha:

`dataframe.to_csv(csv_buffer, header=False, index=False)`

Essa linha grava o DataFrame pandas (que foi passado para a função) no buffer de E/S chamado *csv_buffer*. Você usa um buffer porque não precisa gravar o arquivo localmente.

Para impedir que os cabeçalhos de coluna sejam gravados, use `header=False`. Para impedir que o índice de pandas seja gerado, use `index=False`.

Para gravar o csv_buffer no Amazon S3 como um objeto, use a operação `put` no `object`, que é uma propriedade do `bucket`.


In [25]:
bucket='c117523a2804658l6882075t1w141574630627-labbucket-dsdghnoxlbeo'

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

import os

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False)
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

Use a função que você criou para fazer upload dos três conjuntos de dados (datasets)

In [26]:
upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

NoCredentialsError: Unable to locate credentials

# Etapa 3: Como treinar o modelo

Agora que os dados estão no Amazon S3, você pode treinar um modelo. 

A primeira etapa é obter o URI do contêiner XGBoost.

In [ ]:
import boto3
from sagemaker.image_uris import retrieve
container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

Em seguida, você deve definir alguns *hiperparâmetros* para o modelo. Como esta é a primeira vez que você está treinando o modelo, você pode usar alguns valores para começar.

In [ ]:
hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

Use a função **estimator** (estimador) para configurar o modelo. Aqui estão alguns parâmetros de interesse:

- **train_instance_count** - Define quantas instâncias serão usadas para treinamento. Você usará a instância *one* (um).
- **train_instance_type** - Define o tipo de instância para treinamento. Nesse caso, é *ml.m4.xlarge*.


In [ ]:
import sagemaker
s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

O estimador precisa de *channels* (canais) para alimentar dados no modelo. Para treinamento, *train_channel* e *validate_channel* serão usados.

In [ ]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

A execução de **fit** treinará o modelo.

**Observação:** Esse processo pode demorar até 5 minutos.

In [ ]:
xgb_model.fit(inputs=data_channels, logs=False)

Após a conclusão do treinamento, você estará pronto para testar e avaliar o modelo. No entanto, você fará testes e validação em laboratórios posteriores.

# Parabéns!

Você concluiu este laboratório e agora pode encerrá-lo seguindo as instruções do guia do laboratório.